In [ ]:
import requests
import time
import uuid
from urllib.parse import urlparse, parse_qs

In [ ]:
OKTA_DOMAIN = "" # xxxx.okta.com
USER_NAME = "" #okta user not email (the hting you use to login in the website)
USER_PWD = "" #pwd

In [ ]:
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/115.0"

HEADERS = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "User-Agent": USER_AGENT
}

def authenticate(session):
    response = session.post(
        f"https://{OKTA_DOMAIN}/api/v1/authn",
        headers=HEADERS,
        json={
            "username": USER_NAME,
            "password": USER_PWD
        },
        verify=False
    )
    response_data = response.json()
    state_token = response_data.get("stateToken")
    push_factor = next(factor for factor in response_data["_embedded"]["factors"] if factor["factorType"] == "push")
    factor_id = push_factor["id"]
    return state_token, factor_id

In [ ]:
def verify_push(session, factor_id, state_token):
    response = session.post(
        f"https://{OKTA_DOMAIN}/api/v1/authn/factors/{factor_id}/verify",
        headers=HEADERS,
        json={
            "stateToken": state_token
        },
        verify=False
    )
    return response.json()

In [ ]:
def token_redirect(session, state_token):
    response = session.get(
        f"https://{OKTA_DOMAIN}/login/token/redirect",
        headers={"User-Agent": USER_AGENT},
        params={ "stateToken": state_token },
        allow_redirects=False,
        verify=False
    )

    response.raise_for_status()
    return response

In [ ]:
def authorize(session):
    params = {
        "client_id": "okta.2b1959c8-bcc0-56eb-a589-cfcfb7422f26",
        "code_challenge": "qjrzSW9gMiUgpUvqgEPE4_-8swvyCtfOVvg55o5S_es",
        "code_challenge_method": "S256",
        "nonce": uuid.uuid4(),
        "redirect_uri": f"https://{OKTA_DOMAIN}/enduser/callback",
        "response_type": "code",
        "state": uuid.uuid4(),
        "scope": "openid profile email okta.users.read.self okta.enduser.dashboard.read"
    }
    
    response = session.get(
        f"https://{OKTA_DOMAIN}/oauth2/v1/authorize",
        headers={"User-Agent": USER_AGENT},
        params=params,
        allow_redirects=False  # prevents the redirect
    )
    
    location_url = response.headers.get("location")
    if location_url:
        parsed_url = urlparse(location_url)
        query_params = parse_qs(parsed_url.query)
        return query_params.get("code")[0]
    return None

In [ ]:
def get_access_token(session, code):
    data = {
        "client_id": "okta.2b1959c8-bcc0-56eb-a589-cfcfb7422f26",
        "redirect_uri": f"https://{OKTA_DOMAIN}/enduser/callback",
        "grant_type": "authorization_code",
        "code_verifier": "M25iVXpKU3puUjFaYWg3T1NDTDQtcW1ROUY5YXlwalNoc0hhakxifmZHag",
        "code": code
    }
    
    response = session.post(
        f"https://{OKTA_DOMAIN}/oauth2/v1/token",
        headers={
            "User-Agent": USER_AGENT,
            "Accept": "application/json",
            "Content-Type": "application/x-www-form-urlencoded"
        },
        data=data
    )
    token_data = response.json()
    return token_data.get("access_token")  

In [ ]:
def get_accounts(token):
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "User-Agent": USER_AGENT,
        "Authorization": f"Bearer {token}"
    }
    
    response = requests.get(
        f"https://{OKTA_DOMAIN}/api/v1/users/me/appLinks",
        headers=headers
    )
    return response.json()

def extract_amazon_aws_links(data):
    aws_links = {}
    for entry in data:
        if entry['appName'] == 'amazon_aws':
            link = entry['linkUrl']
            label = entry['label']
            aws_links[link] = label
    return aws_links

In [ ]:
with requests.Session() as session:
    authorize(session) # this call creates cookies needed by the token_redirect
    
    state_token, factor_id = authenticate(session)

    result = verify_push(session, factor_id, state_token)

    time.sleep(10)

    result = verify_push(session, factor_id, state_token)

    print("authenticated !")

    pre_authz_cookies = token_redirect(session, state_token)
    auth_code = authorize(session)
    access_token = get_access_token(session, auth_code)
    accounts_data = get_accounts(access_token)
    aws_links = extract_amazon_aws_links(accounts_data)
        
    print("Amazon AWS links:")
    [print(key,':',value) for key, value in aws_links.items()]